In [1]:
import sys
import numpy as np
import tensorflow as tf
from PyQt6.QtWidgets import (
    QApplication, QWidget, QLabel, QPushButton, QFileDialog, QVBoxLayout,
    QHBoxLayout, QMainWindow, QGraphicsView, QGraphicsScene
)
from PyQt6.QtGui import QPainter, QPen, QPixmap, QColor, QImage
from PyQt6.QtCore import Qt, QPoint
import cv2
from PIL import Image
from matplotlib import pyplot as plt
from pathlib import Path
import random
import os

2025-09-18 23:03:12.478567: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-18 23:03:12.554749: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758207792.612907   25429 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758207792.626804   25429 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758207792.685728   25429 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
IMG_SIZE = 224

class Canvas(QWidget):
    def __init__(self):
        print("start canvas init...")
        super().__init__()
        self.setFixedSize(IMG_SIZE, IMG_SIZE)
        self.image = QPixmap(self.size())
        self.image.fill(Qt.GlobalColor.white)
        self.drawing = False
        self.last_point = QPoint()
        self.pen = QPen(Qt.GlobalColor.black, 4, Qt.PenStyle.SolidLine)
        self.undo_stack = []

    def paintEvent(self, event):
        canvas_painter = QPainter(self)
        canvas_painter.drawPixmap(self.rect(), self.image, self.image.rect())

    def mousePressEvent(self, event):
        if event.button() == Qt.MouseButton.LeftButton:
            self.undo_stack.append(self.image.copy())
            self.drawing = True
            self.last_point = event.position().toPoint()

    def mouseMoveEvent(self, event):
        if self.drawing:
            painter = QPainter(self.image)
            painter.setPen(self.pen)
            painter.drawLine(self.last_point, event.position().toPoint())
            self.last_point = event.position().toPoint()
            self.update()

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.MouseButton.LeftButton:
            self.drawing = False

    def clear(self):
        self.image.fill(Qt.GlobalColor.white)
        self.update()

    def undo(self):
        if self.undo_stack:
            self.image = self.undo_stack.pop()
            self.update()

    def load_image(self, path):
        self.image.load(path)
        self.image = self.image.scaled(self.size())
        self.update()
    
    def get_image_array(self):
    # Resize to 224x224 and convert to RGB888
        image = self.image.toImage().scaled(224, 224).convertToFormat(QImage.Format.Format_RGB888)
        buffer = image.bits()
        buffer.setsize(image.sizeInBytes())
        arr = np.frombuffer(buffer, dtype=np.uint8).reshape((224, 224, 3))
        return arr.astype(np.float32) # / 255.0 vgg19 didn't do normalization





In [3]:
def generate_images(model, test_input):
  prediction = model(test_input, training=True)
  plt.figure(figsize=(15, 15))

  display_list = [test_input[0], prediction[0]]
  title = ['Input Image', 'Predicted Image']

  tmp_path = Path("/mnt/c/Users/jone9/Documents/Code_Project/college/junior/Topic/output/temp_output.png")

  for i in range(2):
    plt.subplot(1, 2, i+1)
    plt.title(title[i])
    # Getting the pixel values in the [0, 1] range to plot.
    show_img = display_list[i] * 0.5 + 0.5
    plt.imshow(show_img)
    plt.axis('off')
    plt.imsave(tmp_path, show_img)
  plt.show()

In [4]:
class MainApp(QMainWindow):
    def __init__(self):
        super().__init__()
        print("start mainapp init...")
        self.setWindowTitle("Cat/Dog Generator")
        self.canvas = Canvas()
        self.label_output = QLabel("...")
        self.label_output_img = QLabel()
        self.label_output_img.setFixedSize(IMG_SIZE, IMG_SIZE)

        self.btn_process = QPushButton("process")
        self.btn_clear = QPushButton("clear")
        self.btn_undo = QPushButton("undo")
        self.btn_random = QPushButton("random")
        self.btn_upload = QPushButton("upload")

        self.btn_process.clicked.connect(self.process)
        self.btn_clear.clicked.connect(self.canvas.clear)
        self.btn_undo.clicked.connect(self.canvas.undo)
        self.btn_random.clicked.connect(self.load_random_image)
        self.btn_upload.clicked.connect(self.upload_image)

        layout_buttons = QHBoxLayout()
        layout_buttons.addWidget(self.btn_undo)
        layout_buttons.addWidget(self.btn_clear)
        layout_buttons.addWidget(self.btn_random)
        layout_buttons.addWidget(self.btn_process)
        layout_buttons.addWidget(self.btn_upload)

        layout_main = QHBoxLayout()
        layout_main.addWidget(self.canvas)
        layout_main.addLayout(layout_buttons)
        layout_main.addWidget(self.label_output_img)

        wrapper = QVBoxLayout()
        wrapper.addLayout(layout_main)
        wrapper.addWidget(self.label_output)

        container = QWidget()
        container.setLayout(wrapper)
        self.setCentralWidget(container)
        
        print("開始載入 VGG19 模型...")
        self.vgg_model = tf.keras.models.load_model("/mnt/c/Users/jone9/Documents/Code_Project/college/junior/Topic/model/vgg19_transfer_model.h5")
        print("已載入 VGG19 模型")

        print("開始載入 Cat Generator 模型...")
        self.cat_gen = tf.keras.models.load_model("/mnt/c/Users/jone9/Documents/Code_Project/college/junior/Topic/model/cat_pix2pix_generator_model.h5")
        print("已載入 Cat Generator 模型")

        print("開始載入 Dog Generator 模型...")
        self.dog_gen = tf.keras.models.load_model("/mnt/c/Users/jone9/Documents/Code_Project/college/junior/Topic/model/dog_pix2pix_generator_model.h5")
        print("已載入 Dog Generator 模型")
    def preprocess_input(self, rgb_img):
        return rgb_img.reshape((1, IMG_SIZE, IMG_SIZE, 3))  # ✅ RGB 3 channel


    def process(self):
        print("click process...")
        self.label_output.setText("...")
        img = self.canvas.get_image_array()
        input_tensor = self.preprocess_input(img)

        preds = self.vgg_model.predict(input_tensor)[0]
        label = np.argmax(preds)
        confidence = preds[label] * 100
        
        print("condidence : ", confidence)
        if label == 0:
            label_text = "cat"  
            generator = self.cat_gen
            self.label_output.setText(f"{label_text} (confidence: {confidence:.2f}%)")
        elif label == 1:
            label_text = "dog"
            generator = self.dog_gen
            self.label_output.setText(f"{label_text} (confidence: {confidence:.2f}%)")
        elif label == 2:
            label_text = "other"
            self.label_output.setText(f"{label_text} (confidence: {confidence:.2f}%)")
            return 
        print(label_text)

        print("resize 244->256")
        
        input_tensor = self.canvas.image.toImage().scaled(256, 256).convertToFormat(QImage.Format.Format_RGB888)
        buffer = input_tensor.bits()
        buffer.setsize(input_tensor.sizeInBytes())
        arr = np.frombuffer(buffer, dtype=np.uint8).reshape((256, 256, 3))
        input_tensor = arr.astype(np.float32) / 255.0
        input_tensor = input_tensor.reshape((1, 256, 256, 3))  # for generator

        print("start pix2pix...")
        output = generator.predict(input_tensor)[0]

        print("output in console")
        generate_images(generator, input_tensor)
        
        tmp_path = Path("/mnt/c/Users/jone9/Documents/Code_Project/college/junior/Topic/output/temp_output.png")
        # Step 1: 把範圍從 [-1, 1] 線性轉成 [0, 1]
        output_norm = (output + 1) / 2

        # Step 2: clip 防止超過界限
        output_norm = np.clip(output_norm, 0, 1)

        # Step 3: 轉成 uint8 (0~255)
        output_uint8 = (output_norm * 255).astype(np.uint8)

        print("shape:", output_uint8.shape, "dtype:", output_uint8.dtype)
        print("min/max:", output_uint8.min(), output_uint8.max())

        # Step 4: 存圖用 plt
        # plt.imsave(tmp_path, output_uint8)

        # output = np.clip(output * 255, 0, 255).astype(np.uint8)
        # plt.imsave(tmp_path, output)

        # 讀檔顯示
        pixmap = QPixmap(str(tmp_path))
        self.label_output_img.setPixmap(pixmap)

    def upload_image(self):
        file_path, _ = QFileDialog.getOpenFileName(self, "upload", "", "Images (*.png *.jpg *.jpeg)")
        if file_path:
            self.canvas.load_image(file_path)

    def load_random_image(self):
        self.folder_path = r'/mnt/c/Users/jone9/Documents/Code_Project/college/junior/Topic/dataset/testedge/'
        self.image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']
        images = [f for f in os.listdir(self.folder_path) if os.path.splitext(f)[1].lower() in self.image_extensions]
        if images:
            chosen = random.choice(images)
            file_path = os.path.join(self.folder_path, chosen)
            self.canvas.load_image(file_path)
        else:
            self.label.setText("資料夾沒有圖片檔案")


In [5]:
app = QApplication(sys.argv)
mainWin = MainApp()
mainWin.show()
sys.exit(app.exec())

start mainapp init...
start canvas init...
開始載入 VGG19 模型...


I0000 00:00:1758207802.915200   25429 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


已載入 VGG19 模型
開始載入 Cat Generator 模型...


已載入 Cat Generator 模型
開始載入 Dog Generator 模型...


已載入 Dog Generator 模型


SystemExit: 0

/home/handsomeguy/anaconda3/envs/py310/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
